<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-03 12:00:38
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.40 C
-------------------
Today PnL: 3.28 L (2.41%)
Current PnL: -29.88 L (-19.06%)
CY Booked + Current PnL: -15.14 L (-9.66%)
-------------------
Total profit:  1.61 L
Total loss:  -31.49 L
-------------------
Total Booked + Current PnL: 11.84 L (9.12%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.56%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 95.39 L (68.32%)
Deployed:  1.30 C
Current:  1.40 C
CAGR/XIRR %: 3.81%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,0.47,4.64,13.94,19.23,21946.0,6974.0,157435.0,484.83,22.81,52.0,L-LC,12.50,182.0,0.32,1.13,26.90,XY25,ATH,MINING
50,MASFIN,1.81,-3.72,26.76,22.05,25244.0,-3645.0,94335.0,398.61,-16.89,50.0,H-SC,3.47,168.0,-0.14,0.68,37.12,XR,ATH,FINANCE
77,TTKPRESTIG,0.64,-27.00,37.11,0.09,27302.0,-27207.0,73570.0,770.00,70.76,35.0,M-SC,7.11,253.0,-1.00,0.53,0.64,OX40N,NTT,DURABLES
33,HCLTECH,2.58,11.62,10.94,23.83,29542.0,28116.0,270040.0,1908.19,16.57,60.0,X-LC,7.71,13.0,0.95,1.93,29.56,X40,ATH,IT
51,MEDANTA,-2.65,-7.32,33.87,24.08,39851.0,-9290.0,117660.0,1486.00,-14.79,48.0,X-SC,10.33,91.0,-0.23,0.84,7.77,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,VAIBHAVGBL,11.31,-23.27,109.51,60.77,153588.0,-42525.0,140250.0,521.00,61.98,64.0,H-SC,4.47,158.0,-0.28,1.00,27.15,XR,NTT,JEWELLERY
76,TRIDENT,11.24,-23.19,67.31,28.51,47699.0,-21390.0,70864.0,48.00,-8.99,67.0,M-SC,10.74,226.0,-0.45,0.51,21.26,XR,NTT,TEXTILES
5,ANGELONE,10.00,8.17,14.83,24.20,48965.0,24925.0,330175.0,3033.00,53.88,56.0,X-SC,1.89,99.0,0.51,2.37,38.28,X40N,NTT,FINANCE
46,JIOFIN,8.95,-14.46,45.52,24.48,104112.0,-38648.0,228717.0,387.00,-15.87,44.0,X-LC,8.49,37.0,-0.37,1.64,32.57,XY24,BTT,FINANCE
23,DIXON,6.65,-23.55,71.69,31.26,126484.0,-54336.0,176432.0,18931.72,-64.29,45.0,X-MC,27.77,56.0,-0.43,1.26,8.28,X40N,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,JCHAC,-3.77,-42.19,72.96,-0.01,53907.0,-53919.0,73886.0,2282.0,15030.43,39.0,M-SC,15.50,235.0,-1.00,0.53,2.44,OX40N,BTT,AC
51,MEDANTA,-2.65,-7.32,33.87,24.08,39851.0,-9290.0,117660.0,1486.0,-14.79,48.0,X-SC,10.33,91.0,-0.23,0.84,7.77,XY24,NTT,HEALTHCARE
44,ITC,-2.31,-22.87,43.63,10.78,80872.0,-54960.0,185358.0,452.0,-54.23,20.0,X-LC,18.18,1.0,-0.68,1.33,0.00,X40,NTT,FMCG
48,KANSAINER,-1.57,-23.07,47.51,13.47,98559.0,-62217.0,207450.0,340.0,-68.31,49.0,H-SC,5.10,159.0,-0.63,1.49,5.48,XY24,NTT,PAINTS
70,SYMPHONY,-0.98,-28.91,40.63,-0.03,49430.0,-49481.0,121660.0,1306.0,-42.85,62.0,M-SC,12.47,206.0,-1.00,0.87,12.43,OX40N,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,2.62,0.87,19.70,20.74,40371.0,1760.0,204930.0,446.0,33.49,55.0,M-MC,4.53,189.0,0.04,1.47,11.57,XY25,NTT,FINANCE
84,WIPRO,2.28,-0.62,73.34,72.27,123503.0,-1050.0,168398.0,420.0,-14.70,46.0,M-LC,2.57,101.0,-0.01,1.21,8.24,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,2.23,-6.40,122.13,107.91,168775.0,-9451.0,138193.0,1641.55,-23.79,48.0,M-SC,11.28,220.0,-0.06,0.99,3.97,OX40N,ATH,CABLES
72,TATAELXSI,1.58,-20.36,65.78,32.03,69065.0,-26841.0,104994.0,9161.00,-13.85,55.0,H-SC,6.49,157.0,-0.39,0.75,17.60,OX40N,NTT,IT
66,SIS,-0.18,-22.41,56.28,21.26,48486.0,-24880.0,86152.0,528.00,2034.72,57.0,H-SC,4.99,163.0,-0.51,0.62,16.40,OX40N,NTT,MISC
48,KANSAINER,-1.57,-23.07,47.51,13.47,98559.0,-62217.0,207450.0,340.00,-68.31,49.0,H-SC,5.10,159.0,-0.63,1.49,5.48,XY24,NTT,PAINTS
40,INDIGOPNTS,0.25,-26.48,36.04,0.02,46254.0,-46219.0,128340.0,1408.00,95.87,31.0,M-SC,5.24,240.0,-1.00,0.92,11.63,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,2.62,0.87,19.70,20.74,40371.0,1760.0,204930.0,446.00,33.49,55.0,M-MC,4.53,189.0,0.04,1.47,11.57,XY25,NTT,FINANCE
18,COALINDIA,0.47,4.64,13.94,19.23,21946.0,6974.0,157435.0,484.83,22.81,52.0,L-LC,12.50,182.0,0.32,1.13,26.90,XY25,ATH,MINING
1,ABB,4.86,5.19,38.38,45.57,105625.0,13589.0,275208.0,7934.00,-32.38,71.0,H-MC,6.03,121.0,0.13,1.97,22.21,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,4.86,5.19,38.38,45.57,105625.0,13589.0,275208.0,7934.00,-32.38,71.0,H-MC,6.03,121.0,0.13,1.97,22.21,AR,NTT,ELECTRICAL
84,WIPRO,2.28,-0.62,73.34,72.27,123503.0,-1050.0,168398.0,420.00,-14.70,46.0,M-LC,2.57,101.0,-0.01,1.21,8.24,XR,NTT,IT
39,INDIAMART,2.35,-2.80,112.67,106.72,135076.0,-3450.0,119886.0,4810.62,-51.96,56.0,H-SC,8.92,138.0,-0.03,0.86,19.93,AR,ATH,MISC
26,FINCABLES,2.23,-6.40,122.13,107.91,168775.0,-9451.0,138193.0,1641.55,-23.79,48.0,M-SC,11.28,220.0,-0.06,0.99,3.97,OX40N,ATH,CABLES
85,ZYDUSLIFE,2.86,-4.59,42.15,35.63,84284.0,-9613.0,199961.0,1286.17,-18.07,53.0,H-MC,5.80,119.0,-0.11,1.43,11.32,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-2.31,-22.87,43.63,10.78,80872.0,-54960.0,185358.0,452.00,-54.23,20.0,X-LC,18.18,1.0,-0.68,1.33,0.00,X40,NTT,FMCG
6,ASIANPAINT,0.75,-15.41,42.99,20.95,91552.0,-38808.0,212960.0,3460.25,-16.23,26.0,X-LC,10.22,36.0,-0.42,1.53,14.73,X40,ATH,PAINTS
13,BERGEPAINT,1.04,-16.09,44.37,21.14,84638.0,-36584.0,190755.0,680.00,-22.05,27.0,X-MC,7.79,75.0,-0.43,1.37,1.75,XY24,NTT,PAINTS
12,BATAINDIA,0.49,-44.02,141.53,35.20,101940.0,-56643.0,72027.0,2096.00,-5.56,31.0,X-SC,20.42,93.0,-0.56,0.52,2.36,X40,NTT,FOOTWEAR
53,PGHH,0.71,-11.05,50.85,34.19,96582.0,-23584.0,189936.0,17907.65,-37.47,33.0,X-MC,8.04,60.0,-0.24,1.36,1.00,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,2.10,-11.02,33.23,18.55,111841.0,-41672.0,336565.0,4311.59,-23.30,58.0,X-LC,0.66,3.0,-0.37,2.41,14.50,X40,ATH,IT
81,VBL,4.09,-1.96,38.38,35.67,118845.0,-6189.0,309653.0,671.64,-13.58,52.0,X-LC,1.59,5.0,-0.05,2.22,11.62,X40N,ATH,FMCG
5,ANGELONE,10.00,8.17,14.83,24.20,48965.0,24925.0,330175.0,3033.00,53.88,56.0,X-SC,1.89,99.0,0.51,2.37,38.28,X40N,NTT,FINANCE
34,HINDUNILVR,0.98,-7.40,22.75,13.67,54156.0,-19013.0,238050.0,2922.00,-14.28,52.0,X-LC,2.26,9.0,-0.35,1.71,12.38,XY25,NTT,FMCG
21,DABUR,0.33,-2.12,46.81,43.70,113662.0,-5253.0,242815.0,735.00,-10.16,42.0,X-MC,2.75,73.0,-0.05,1.74,12.61,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-2.31,-22.87,43.63,10.78,80872.0,-54960.0,185358.0,452.00,-54.23,20.0,X-LC,18.18,1.0,-0.68,1.33,0.00,X40,NTT,FMCG
53,PGHH,0.71,-11.05,50.85,34.19,96582.0,-23584.0,189936.0,17907.65,-37.47,33.0,X-MC,8.04,60.0,-0.24,1.36,1.00,X40,ATH,FMCG
13,BERGEPAINT,1.04,-16.09,44.37,21.14,84638.0,-36584.0,190755.0,680.00,-22.05,27.0,X-MC,7.79,75.0,-0.43,1.37,1.75,XY24,NTT,PAINTS
12,BATAINDIA,0.49,-44.02,141.53,35.20,101940.0,-56643.0,72027.0,2096.00,-5.56,31.0,X-SC,20.42,93.0,-0.56,0.52,2.36,X40,NTT,FOOTWEAR
3,ACC,2.75,-29.36,132.36,64.15,222497.0,-69851.0,168100.0,3906.00,-58.50,44.0,X-SC,7.56,84.0,-0.31,1.20,2.75,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,1.31,-28.66,100.23,42.84,46482.0,-18631.0,46375.0,424.00,-53.17,55.0,X-SC,21.67,83.0,-0.40,0.33,9.01,XY24,NTT,MISC
12,BATAINDIA,0.49,-44.02,141.53,35.20,101940.0,-56643.0,72027.0,2096.00,-5.56,31.0,X-SC,20.42,93.0,-0.56,0.52,2.36,X40,NTT,FOOTWEAR
58,RELAXO,1.54,-49.77,203.21,52.31,148173.0,-72244.0,72916.0,1176.00,-46.57,51.0,X-SC,11.60,92.0,-0.49,0.52,8.43,X40N,NTT,FOOTWEAR
51,MEDANTA,-2.65,-7.32,33.87,24.08,39851.0,-9290.0,117660.0,1486.00,-14.79,48.0,X-SC,10.33,91.0,-0.23,0.84,7.77,XY24,NTT,HEALTHCARE
35,HONAUT,0.04,-17.42,73.55,43.32,98925.0,-28372.0,134500.0,58357.33,-29.05,54.0,X-SC,8.83,90.0,-0.29,0.96,9.21,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-2.31,-22.87,43.63,10.78,80872.0,-54960.0,185358.0,452.00,-54.23,20.0,X-LC,18.18,1.0,-0.68,1.33,0.00,X40,NTT,FMCG
75,TMPV,3.31,-23.16,60.04,22.98,104217.0,-52319.0,173579.0,600.00,-20.63,65.0,X-LC,4.38,2.0,-0.50,1.24,10.97,XY24,NTT,AUTO
73,TCS,2.10,-11.02,33.23,18.55,111841.0,-41672.0,336565.0,4311.59,-23.30,58.0,X-LC,0.66,3.0,-0.37,2.41,14.50,X40,ATH,IT
81,VBL,4.09,-1.96,38.38,35.67,118845.0,-6189.0,309653.0,671.64,-13.58,52.0,X-LC,1.59,5.0,-0.05,2.22,11.62,X40N,ATH,FMCG
42,INFY,2.95,14.20,17.56,34.24,62743.0,44420.0,357308.0,1972.00,-10.98,58.0,X-LC,7.28,6.0,0.71,2.56,23.65,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,2.51,-31.11,92.92,32.89,49419.0,-24022.0,53184.0,1800.00,-756.83,62.0,L-MC,11.28,259.0,-0.49,0.38,46.46,XR,NTT,BANKS
67,SONACOMS,5.75,-8.62,52.22,39.09,48270.0,-8724.0,92435.0,804.02,-28.50,73.0,M-SC,2.37,227.0,-0.18,0.66,30.77,AR,ATH,AUTO
14,BSOFT,4.99,-19.00,86.15,50.79,98154.0,-26719.0,113934.0,831.70,3.98,62.0,H-SC,6.16,171.0,-0.27,0.82,32.66,XR,ATH,IT
79,VAIBHAVGBL,11.31,-23.27,109.51,60.77,153588.0,-42525.0,140250.0,521.00,61.98,64.0,H-SC,4.47,158.0,-0.28,1.00,27.15,XR,NTT,JEWELLERY
18,COALINDIA,0.47,4.64,13.94,19.23,21946.0,6974.0,157435.0,484.83,22.81,52.0,L-LC,12.50,182.0,0.32,1.13,26.90,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,4.86,5.19,38.38,45.57,105625.0,13589.0,275208.0,7934.00,-32.38,71.0,H-MC,6.03,121.0,0.13,1.97,22.21,AR,NTT,ELECTRICAL
5,ANGELONE,10.00,8.17,14.83,24.20,48965.0,24925.0,330175.0,3033.00,53.88,56.0,X-SC,1.89,99.0,0.51,2.37,38.28,X40N,NTT,FINANCE
67,SONACOMS,5.75,-8.62,52.22,39.09,48270.0,-8724.0,92435.0,804.02,-28.50,73.0,M-SC,2.37,227.0,-0.18,0.66,30.77,AR,ATH,AUTO
33,HCLTECH,2.58,11.62,10.94,23.83,29542.0,28116.0,270040.0,1908.19,16.57,60.0,X-LC,7.71,13.0,0.95,1.93,29.56,X40,ATH,IT
27,GILLETTE,1.55,4.18,27.10,32.42,71682.0,10614.0,264510.0,11206.78,-14.26,66.0,X-SC,12.37,87.0,0.15,1.90,19.91,X40,ATH,FMCG


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.96
1,25,44.93
2,50,76.72


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.18
MC    30.43
LC    24.37
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.25
X40      23.82
X40N     15.14
AR        9.00
XR        8.83
XY25      8.63
OX40N     7.38
SR        0.93
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.74
H-SC    23.01
X-LC    20.87
M-SC    11.06
X-SC    10.37
H-MC     4.84
M-MC     1.47
M-LC     1.21
H-LC     1.16
L-LC     1.13
L-SC     0.74
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.04,-12.87,51.35
FINANCE,13.36,-12.16,56.33
IT,12.95,-18.14,77.15
MISC,6.97,-33.45,88.04
ELECTRICAL,6.08,-10.53,50.86
PAINTS,5.31,-24.86,43.41
INSURANCE,4.78,-1.56,36.31
PHARMA,4.01,-4.86,37.86
AUTO,2.95,-26.92,70.81


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3306756.0,21
AR,1313871.0,10
XR,1259340.0,12
X40,1184536.0,15
X40N,1006311.0,10
OX40N,761901.0,10
XY25,415462.0,6
SR,290713.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3584215.0,24
X-MC,1715873.0,16
M-SC,1512836.0,15
X-LC,978928.0,12
X-SC,850249.0,9
H-MC,402746.0,3
L-SC,187275.0,2
M-LC,123503.0,1
H-LC,71529.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1231615.0      6
           AR         929907.0      5
M-SC       XY24       861246.0      6
H-SC       XR         808098.0      7
X-MC       X40        574103.0      7
           XY24       483899.0      3
X-LC       X40        436811.0      6
X-MC       X40N       423876.0      4
X-SC       X40N       367797.0      4
M-SC       OX40N      345668.0      5
H-SC       OX40N      323882.0      4
X-SC       XY24       308830.0      3
H-SC       SR         290713.0      2
X-MC       XY25       233995.0      2
X-LC       X40N       214638.0      2
H-MC       XY24       212837.0      1
X-LC       XY24       208329.0      2
H-MC       AR         189909.0      2
M-SC       XR         183396.0      2
X-SC       X40        173622.0      2
M-LC       XR         123503.0      1
M-SC       AR         122526.0      2
X-LC       XY25       119150.0      2
L-SC       XR          94924.0      1
           OX40N       92351.0      1
H-LC       AR          71529.0      1
L-MC       XR          49419.0      1
M-MC       XY25        40371.0      1
L-LC       XY25        21946.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
